# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import the weather_data.csv file as a DataFrame
weather_data = pd.read_csv("../Resources/weather_data.csv", encoding="utf-8")
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bathsheba,13.2167,-59.5167,78.80,73,40,21.85,BB,1612061852
1,Avarua,-21.2078,-159.7750,84.20,70,75,4.61,CK,1612061852
2,Jamestown,42.0970,-79.2353,18.00,79,1,5.75,US,1612061821
3,Richards Bay,-28.7830,32.0377,74.25,94,85,4.36,ZA,1612061646
4,Rikitea,-23.1203,-134.9692,79.38,72,0,11.52,PF,1612061852


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
no_clouds = weather_data.loc[weather_data["Cloudiness"] == 0, :]

In [6]:
low_wind = no_clouds.loc[no_clouds["Wind Speed"] <= 10, :]

In [7]:
ideal_weather = low_wind.loc[(low_wind["Max Temp"] <= 80) & (low_wind["Max Temp"] >= 70), :]

In [8]:
hotel_df = ideal_weather.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
169,Saint-Pierre,-21.3393,55.4781,79.00,65,0,5.75,RE,1612061871
213,Esperance,-33.8667,121.9000,78.80,41,0,6.91,AU,1612061875
228,Saint-Philippe,-21.3585,55.7679,79.00,65,0,5.75,RE,1612061877
233,Bānāvar,13.4136,76.1683,70.54,55,0,6.64,IN,1612061877
350,Montes Claros,-16.7350,-43.8617,77.00,50,0,3.31,BR,1612061892
360,Gambēla,8.2500,34.5833,75.99,44,0,2.75,ET,1612061893
458,Juba,4.8517,31.5825,72.37,24,0,2.86,SS,1612061904


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df.insert(8, "Hotel_Name", True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel_Name,Date
169,Saint-Pierre,-21.3393,55.4781,79.00,65,0,5.75,RE,True,1612061871
213,Esperance,-33.8667,121.9000,78.80,41,0,6.91,AU,True,1612061875
228,Saint-Philippe,-21.3585,55.7679,79.00,65,0,5.75,RE,True,1612061877
233,Bānāvar,13.4136,76.1683,70.54,55,0,6.64,IN,True,1612061877
350,Montes Claros,-16.7350,-43.8617,77.00,50,0,3.31,BR,True,1612061892
360,Gambēla,8.2500,34.5833,75.99,44,0,2.75,ET,True,1612061893
458,Juba,4.8517,31.5825,72.37,24,0,2.86,SS,True,1612061904


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "Hotel",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/terrilvallikalam/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing field/result... skipping.


In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel_Name,Date
169,Saint-Pierre,-21.3393,55.4781,79.00,65,0,5.75,RE,LODGE PALMAE,1612061871
213,Esperance,-33.8667,121.9000,78.80,41,0,6.91,AU,Esperance Chalet Village,1612061875
228,Saint-Philippe,-21.3585,55.7679,79.00,65,0,5.75,RE,Les Embruns Du Baril,1612061877
233,Bānāvar,13.4136,76.1683,70.54,55,0,6.64,IN,True,1612061877
350,Montes Claros,-16.7350,-43.8617,77.00,50,0,3.31,BR,ibis Montes Claros Shopping,1612061892
360,Gambēla,8.2500,34.5833,75.99,44,0,2.75,ET,BARO Gambella (ETHIOPIA) hotel,1612061893
458,Juba,4.8517,31.5825,72.37,24,0,2.86,SS,Pyramid Continental Hotel,1612061904


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add markers to heat map
marker_layer = gmaps.marker_layer(
    locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))